In [43]:
from graph_tool.all import *
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import time


In [3]:
X_train_raw = pd.read_csv('InputData/xtrain_challenge.csv')
y_train_raw = pd.read_csv('InputData/ytrain_challenge.csv')
y_train_raw = np.array(y_train_raw).ravel()
X_test_raw = pd.read_csv('InputData/xtest_challenge.csv')

In [4]:
cols_A= ["fA1","fA2","fA3","fA4","fA5","fA6","fA7",
           "fA8","fA9","fA10","fA11","fA12","fA13","fA14"]
cols_B= ["fB1","fB2","fB3","fB4","fB5","fB6","fB7",
           "fB8","fB9","fB10","fB11","fB12","fB13","fB14"]
cols_s= ["s1","s2","s3","s4","s5","s6","s7","s8"]

In [7]:
def _create_list_of_images_feat(X_raw):
    X = X_raw.copy()
    
    X["featA"] = X["fA1"].astype("str")+"_"+X["fA2"].astype("str")+"_"+ \
    X["fA3"].astype("str")+"_"+X["fA4"].astype("str")+"_"+ \
    X["fA5"].astype("str")+"_"+X["fA6"].astype("str")+"_"+ \
    X["fA7"].astype("str")+"_"+X["fA8"].astype("str")+"_"+ \
    X["fA9"].astype("str")+"_"+X["fA10"].astype("str")+"_"+ \
    X["fA11"].astype("str")+"_"+X["fA12"].astype("str")+"_"+ \
    X["fA13"].astype("str")+"_"+X["fA14"].astype("str")

    X["featB"] = X["fB1"].astype("str")+"_"+X["fB2"].astype("str")+"_"+ \
    X["fB3"].astype("str")+"_"+X["fB4"].astype("str")+"_"+ \
    X["fB5"].astype("str")+"_"+X["fB6"].astype("str")+"_"+ \
    X["fB7"].astype("str")+"_"+X["fB8"].astype("str")+"_"+ \
    X["fB9"].astype("str")+"_"+X["fB10"].astype("str")+"_"+ \
    X["fB11"].astype("str")+"_"+X["fB12"].astype("str")+"_"+ \
    X["fB13"].astype("str")+"_"+X["fB14"].astype("str")

    X.drop(cols_A, axis = 1, inplace = True)
    X.drop(cols_B, axis = 1, inplace = True)
    X.drop(cols_s, axis = 1, inplace = True)
    
    df=pd.concat([X["featA"],X["featB"]],axis = 0)
    
    return df
    

In [8]:
X_train_images = _create_list_of_images_feat(X_train_raw)
X_test_images = _create_list_of_images_feat(X_test_raw)
X_all_images = pd.concat([X_train_images, X_test_images], axis = 0 )

In [15]:
le = LabelEncoder()
le.fit(X_all_images)

LabelEncoder()

In [18]:
def label_encoding(X_raw, le): 
    X = X_raw.copy()
    
    X["featA"] = X["fA1"].astype("str")+"_"+X["fA2"].astype("str")+"_"+ \
    X["fA3"].astype("str")+"_"+X["fA4"].astype("str")+"_"+ \
    X["fA5"].astype("str")+"_"+X["fA6"].astype("str")+"_"+ \
    X["fA7"].astype("str")+"_"+X["fA8"].astype("str")+"_"+ \
    X["fA9"].astype("str")+"_"+X["fA10"].astype("str")+"_"+ \
    X["fA11"].astype("str")+"_"+X["fA12"].astype("str")+"_"+ \
    X["fA13"].astype("str")+"_"+X["fA14"].astype("str")

    X["featB"] = X["fB1"].astype("str")+"_"+X["fB2"].astype("str")+"_"+ \
    X["fB3"].astype("str")+"_"+X["fB4"].astype("str")+"_"+ \
    X["fB5"].astype("str")+"_"+X["fB6"].astype("str")+"_"+ \
    X["fB7"].astype("str")+"_"+X["fB8"].astype("str")+"_"+ \
    X["fB9"].astype("str")+"_"+X["fB10"].astype("str")+"_"+ \
    X["fB11"].astype("str")+"_"+X["fB12"].astype("str")+"_"+ \
    X["fB13"].astype("str")+"_"+X["fB14"].astype("str")

    X.drop(cols_A, axis = 1, inplace = True)
    X.drop(cols_B, axis = 1, inplace = True)
    X.drop(cols_s, axis = 1, inplace = True)
    
    X["idA"] = le.transform(X["featA"])
    X["idB"] = le.transform(X["featB"])

    X.drop(["featA","featB"], axis = 1 , inplace=True)
    
    return X

In [20]:
X_train_lbl = label_encoding(X_train_raw, le)
X_test_lbl = label_encoding(X_test_raw, le)

In [22]:
mask_match = (y_train_raw == 1)
X_train_match = X_train_lbl[mask_match]
X_train_match.drop_duplicates(inplace=True)

/home/mickael/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
g_match = Graph(directed  = False)

In [25]:
g_match.add_edge_list(X_train_match.values)

In [27]:
X_test_lbl["y"] = 0

In [32]:
X_train_match.head()

,idA,idB
5,24518,112373
12,110624,115300
16,116170,88704
17,84056,97034
18,9939,1781


In [46]:
def _check_match(row): 
    match = 0 
    dist = shortest_distance(g_match, source=g_match.vertex(lblA), target = g_match.vertex(lblB))
    if dist < 2147483600: 
        match = 1
    
    row["y"] = match 
    return row


In [51]:
start = time.time()
X_test_lbl.loc[0:10000].apply(_check_match,axis = 1 )
end = time.time()

print("The process took {:.4f} seconds".format(end-start))

The process took 22.0381 seconds


In [52]:
X_test_lbl.shape

(1598219, 3)

In [ ]:
graph_draw(g, vertex_text=g.vertex_index, vertex_font_size=18,
           output_size=(200, 200), output="two-nodes.png")